In [1]:
import os
os.sys.path.append("..")

In [2]:
import torch
from internvl.extern.hf.modeling_internvl import InternVLChatModel
from internvl.extern.hf.tokenization_internlm2 import InternLM2Tokenizer

torch.cuda.set_device(3)

model_name_or_path = "/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/cunxin/internvl_checkpoint/2b"

model = InternVLChatModel.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    local_files_only=True
).eval().cuda()

tokenizer = InternLM2Tokenizer.from_pretrained(
    model_name_or_path,
    local_files_only=True,
    use_fast=False
)

/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/conda/envs/vima/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [4]:
%matplotlib inline
from internvl.extern.hf.processing_internvl import load_image_from_file, process_image
import matplotlib.pyplot as plt
from PIL import Image

generation_config = dict(max_new_tokens=1024, do_sample=True)

image_size = 224

# url1 = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# url2 = 'http://images.cocodataset.org/val2017/000000009769.jpg'
# img1, img2 = requests.get(url1, stream=True).raw, requests.get(url2, stream=True).raw
img1, img2 = "assets/+.jpg", "assets/000000039769.jpg"
img1 = Image.open(img1).convert('RGB')
img2 = Image.open(img2).convert('RGB')
img1 = img1.resize((image_size, image_size))
# img2 = img2.resize((image_size, image_size))
# plt.imshow(img1)
pixel_values = process_image(img1, input_size=image_size).to(torch.bfloat16).cuda()
# pixel_values1, pixel_values2 = load_image_from_file(img1).to(torch.bfloat16).cuda(), load_image_from_file(img2).to(torch.bfloat16).cuda()
# pixel_values1 = pixel_values1[-1:, ...]
# pixel_values2 = pixel_values2[-1:, ...]
# pixel_values = torch.cat([pixel_values1, pixel_values2], dim=0)
# num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]
num_patches_list = [pixel_values.size(0)]
# prompt = f"What do you see separately in the following two images? Image1: <image>\nImage2: <image>"
prompt = f"What do you see separately in the following image? Image: <image>"
response, history = model.chat(tokenizer, pixel_values, prompt, generation_config,
                               num_patches_list=num_patches_list,
                               history=None, return_history=True)
print(response)

Sorry, that's my mistake. What did you see in the image?

![My apologies, as expected.](https://i.imgur.com/9q2Wg9M.png)


In [ ]:
print(model.language_model.get_output_embeddings().weight.data)
print(model.language_model.get_input_embeddings().weight.data)

: 